# Lab: Exploring LLM Agents in LLM Agentic Tool Mesh

## Introduction

In this lab, we'll explore the concept of **LLM Agents** within the LLM Agentic Tool Mesh platform. LLM Agentic Tool Mesh provides all the necessary tools to build a powerful agentic system by handling:

- **Tool Repository**
- **Reasoning Engine**
- **Multi-Agents Task Force**

## Objectives

By the end of this lab, you will:

- Understand the concept of LLM Agents and how they operate within LLM Agentic Tool Mesh.
- Learn the differences between the Star and Snowflake architectures for coordinating multi-agent systems.
- Define and manage tasks for multiple agents using LLM Agentic Tool Mesh's Agent Service.

## Definition of LLM Agents

### What is an Agent?

In the context of LLM, an **agent** is an autonomous entity capable of:

- **Perceiving its environment**: Agents can gather and interpret information from their surroundings.
- **Making decisions**: Based on the information perceived, agents can decide on the best course of action.
- **Acting on decisions**: Agents execute actions that help achieve specific objectives.

These agents can operate independently or interact with one another to optimize their collective performance, depending on the complexity of the task.

### What is Multi-Agent AI?

**Multi-Agent AI** involves coordinating multiple agents, each specialized in a specific domain or function, to collaborate and achieve a common goal. These agents can handle:

- **Task Division**: Agents can divide a complex task into smaller, manageable parts.
- **Specialization**: Each agent may specialize in a particular function, such as information retrieval, summarization, or decision-making.
- **Collaboration**: Agents can communicate and share information, enabling more effective and efficient task execution.

Managing such agents typically requires advanced coding and deep knowledge of agent-based systems. However, **LLM Agentic Tool Mesh simplifies** this process by providing high-level abstraction through intuitive prompts and configuration files. This allows users to focus on defining tasks and desired outcomes, while LLM Agentic Tool Mesh handles the coordination, task distribution, and result aggregation behind the scenes.

## Architecture Options for LLM Agents in LLM Agentic Tool Mesh

LLM Agentic Tool Mesh offers two primary architectural models for managing LLM Agents: the **Star Architecture** and the **Snowflake Architecture**:

In the **Star** architecture, a central Reasoning Engine powered by an LLM orchestrates various tools. This setup enables a single chatbot agent to manage and execute tasks using a suite of individual tools, providing centralized control and streamlined management.

The **Snowflake** architecture expands on the Star model by introducing multiple LLM-equipped agents. These agents collaborate, sharing resources and tasks, which enhances the system's ability to handle complex operations. This distributed approach significantly improves performance through cooperative task execution.

<div align="center">
  <img src="pictures/multiagents.png" alt="Multi Agents" width="800">
</div>

## Getting Started

### Prerequisites

- Python 3.11 installed on your system.
- LLM Agentic Tool Mesh library installed. If not, install it using: `pip install 'llmesh[agents]'`
- Install kernel for Jupiter notebook `pip install ipykernel`
- Create `.env` file inside `data` folder with the value of `LLM_API_KEY` and `LLM_MODEL_NAME` environment variables
- API keys for the LLM services you plan to use (e.g., OpenAI).

### Tool Repository

Agents in LLM Agentic Tool Mesh rely on **tools** to perform specialized tasks such as information retrieval, document summarization, or data analysis. These tools act as extensions to the agents' capabilities, allowing them to efficiently complete complex operations. The effectiveness of the system is greatly enhanced by how well these tools are organized and accessed, making **tool management** a critical aspect of working with multi-agent systems.

The **Tool Repository** service in LLM Agentic Tool Mesh is designed to simplify and automate the storage, management, and retrieval of tools. This service leverages the concepts we explored earlier to ensure that agents have seamless access to a wide range of tools, along with their associated metadata, to execute tasks effectively and efficiently.

#### Key Features

1. **Dynamic Tool Storage**:
   - Allows for adding tools with associated metadata, including tool name, description, function, and usage parameters.
   - Supports a variety of tool types, from pre-built utilities to custom implementations tailored for specific agent tasks.
   - Ensures that tools are organized and readily available for use by multiple agents.

2. **Tool Retrieval Based on Criteria**:
   - Provides flexible search and retrieval functionality, enabling agents to access tools based on specific criteria such as task requirements, tool type, or metadata attributes.
   - Ensures that the right tools are selected for the right tasks, optimizing the performance of the multi-agent system.

3. **Metadata Management**:
   - Stores relevant metadata for each tool, including versioning, author information, last usage, and specific capabilities.
   - Metadata can be queried and used to decide which tools are best suited for certain tasks or environments.

In [1]:
import tqdm as notebook_tqdm
import warnings

# Suppress warnings
warnings.filterwarnings(
    "ignore",
    message=r'Field "model_name" in Config has conflict with protected namespace "model_".*',
    category=UserWarning,
)

In [2]:
from athon.agents import ToolRepository

# Example configuration for the Tool Repository
REPO_CONFIG = {
    'type': 'LangChainStructured'
}

# Initialize the Tool Repository with the provided configuration
tool_repository = ToolRepository.create(REPO_CONFIG)

/Users/antoniofin/Documents/Code/llmesh/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain.tools import tool

# Example tool and metadata to be added to the repository
@tool
def text_summarizer(text: str) -> str:
    """A simple text summarizer function"""
    return text[:50] 

metadata = {
    'category': 'NLP',
    'version': '1.0',
    'author': 'John Doe'
}

# Add the tool to the repository
add_result = tool_repository.add_tool(text_summarizer, metadata)

if add_result.status == "success":
    print("Tool added successfully.")
else:
    print(f"ERROR:\n{add_result.error_message}")

2024-12-17 14:10:52,398 - ATHON - DEBUG - Added tool to repository
Tool added successfully.


In [4]:
# Retrieve tools with a metadata filter
metadata_filter = {'category': 'NLP'}
get_result = tool_repository.get_tools(metadata_filter)

if get_result.status == "success":
    print(f"RETRIEVED TOOLS:\n{get_result.tools}")
else:
    print(f"ERROR:\n{get_result.error_message}")

RETRIEVED TOOLS:
[{'object': StructuredTool(name='text_summarizer', description='A simple text summarizer function', args_schema=<class 'langchain_core.utils.pydantic.text_summarizer'>, func=<function text_summarizer at 0x324a6da80>), 'metadata': {'category': 'NLP', 'version': '1.0', 'author': 'John Doe'}}]


### Reasoning Engine

As we explored earlier, agents in LLM Agentic Tool Mesh can perform complex tasks by leveraging a suite of tools. However, for agents to deliver truly intelligent, context-aware responses, they must coordinate their tools effectively. This is where the **Reasoning Engine** plays a central role. The Reasoning Engine orchestrates interactions between the **LLM** and various tools, enabling agents to seamlessly combine their decision-making capabilities with tool-based actions.

The **Reasoning Engine** in LLM Agentic Tool Meshh extends the chat capabilities by managing the dynamic integration of tools with the LLM, allowing for real-time decision-making and execution of tasks. This service ensures that the LLM-generated responses are not only contextually relevant but also actionable by using the appropriate tools from the Tool Repository.

#### Key Features

1. **Tool Orchestration**:
   - The Reasoning Engine coordinates between the LLM and the tools, deciding which tools to invoke based on the context and user input.
   - Dynamically loads and executes tools based on the task requirements, optimizing the use of each tool within the multi-agent system.

2. **Memory Management**:
   - The Reasoning Engine handles the storage and retrieval of relevant memory for ongoing tasks or conversations, enabling agents to "remember" prior steps and interactions.
   - Supports **memory clearing** and reconfiguration, allowing users to reset or adjust the scope of memory to handle different workflows or scenarios.
   - Provides the ability to manage the agent's memory across sessions, ensuring that relevant context is preserved across conversations or tasks.

3. **Dynamic Configuration**:
   - Allows users to configure and adjust the Reasoning Engine's behavior dynamically, tailoring the interaction between LLMs and tools to specific tasks.
   - Users can modify which tools are loaded, how the tools interact, and how memory is used during task execution, giving fine-grained control over how agents perform in real-time.

<div align="center">
  <img src="pictures/reasoning.png" alt="Reasoning Engine" width="800">
</div>
   

### Task Force

The **Task Force Multi-Agents** service in LLM Agentic Tool Mesh enables the orchestration of complex tasks through a network of specialized agents. This service allows users to define a structured workflow, where each agent is assigned a specific task, and these tasks are executed in sequence or in parallel, depending on the defined methodology. The **Task Force** service ensures that each task is handled efficiently by leveraging multiple agents and their respective tools, all while being orchestrated by the reasoning power of an LLM.

#### Key Features

1. **LLM-Driven Planning**:
   - The Task Force integrates with an **LLM** to plan the sequence of tasks, ensuring that the workflow is intelligently coordinated.
   - The LLM can generate high-level strategies for completing tasks based on user input, and the Task Force orchestrates the execution across agents.

2. **Agent Specialization**:
   - Each agent in the Task Force can be specialized for a particular task, such as data analysis, report summarization, or presentation creation.
   - The behavior of each agent can be tailored through prompts that define their role, backstory, and goals.

3. **Task-Oriented Workflow**:
   - Allows users to define complex, multi-step tasks and assign each step to a dedicated agent.
   - Supports both **sequential** and **parallel** task execution, offering flexibility in how tasks are processed.
   - Users can configure task parameters, expected outputs, and agent behaviors through easy-to-define prompts and configuration files.

4. **Tool Integration**:
   - Agents in the Task Force can utilize a suite of tools, such as data fetchers, summarizers, or presentation builders, to complete their tasks.
   - Tools are assigned to agents based on their roles, and these tools are dynamically loaded and executed during task completion.

In [5]:
from dotenv import load_dotenv
import os

# Specify the path to the .env file
dotenv_path = os.path.join('data', '.env')

# Load environment variables from the .env file located in the Data folder
load_dotenv(dotenv_path)

# Read environment variables
llm_api_key = os.getenv('LLM_API_KEY')
if not llm_api_key:
    raise ValueError("API key is not set in environment variables.")
llm_model = os.getenv('LLM_MODEL_NAME')
if not llm_model:
    raise ValueError("Model name is not set in environment variables.")

In [6]:
from athon.agents import TaskForce

# Configuration for the Task Force Multi-Agents
TASK_FORCE_CONFIG = {
    'type': 'CrewAIMultiAgent',
    'plan_type': 'Sequential',
    'tasks': [
        {
            'description': 'Perform research to gather information for a blog post on {request}.',
            'expected_output': 'A summary of key insights, facts, and trends related to the topic.',
            'agent': {
                'role': 'Research Agent',
                'goal': 'Gather relevant information for the blog post',
                'backstory': 'Expert in researching and summarizing information quickly and accurately',
                'tools': []
            }
        },
        {
            'description': 'Generate a structured outline for a blog post on {request} based on the research.',
            'expected_output': 'A detailed blog post outline with 3-5 main sections.',
            'agent': {
                'role': 'Outline Agent',
                'goal': 'Create a comprehensive blog post outline',
                'backstory': 'Expert in structuring content into engaging and informative blog outlines',
                'tools': []
            }
        },
        {
            'description': 'Develop a complete blog post including an introduction, main content, and conclusion.',
            'expected_output': 'A full blog post of around 500-800 words.',
            'agent': {
                'role': 'Content Development Agent',
                'goal': 'Write a well-researched blog post with intro, body, and conclusion',
                'backstory': 'Skilled at turning outlines into engaging and coherent blog posts',
                'tools': []
            }
        },
        {
            'description': 'Optimize the blog post for SEO by adding keywords, hashtags, and improving readability.',
            'expected_output': 'An SEO-optimized version of the blog post, including keywords, meta descriptions, and hashtags.',
            'agent': {
                'role': 'SEO Agent',
                'goal': 'Enhance the blog post with SEO optimizations',
                'backstory': 'Expert in applying SEO strategies to improve blog visibility and ranking',
                'tools': []
            }
        },
        {
            'description': 'Perform a final check for coherence, clarity, and overall quality of the blog post.',
            'expected_output': 'A polished, ready-to-publish blog post.',
            'agent': {
                'role': 'Final Agent',
                'goal': 'Ensure the blog post is ready for publication',
                'backstory': 'Final quality control expert ensuring the post is polished and publication-ready',
                'tools': []
            }
        }
    ],
    'llm': {
        'type': 'LangChainChatOpenAI',
        'api_key': llm_api_key,
        'model_name': llm_model,
        'temperature': 0.7,
    },
    'verbose': True,
    'memory': False
}

# Initialize the Task Force with the provided configuration
task_force = TaskForce.create(TASK_FORCE_CONFIG)

2024-12-17 14:10:52,414 - ATHON - DEBUG - Selected Langchain ChatOpenAI
2024-12-17 14:10:52,442 - ATHON - DEBUG - Returned model 'gpt-4o-mini'


In [7]:
# Run the task force with an input message
input_message = "Write a blog post about the importance of renewable energy."
result = task_force.run(input_message)

# Handle the response
if result.status == "success":
    print(f"COMPLETION:\n{result.completion}")
else:
    print(f"ERROR:\n{result.error_message}")

# Agent: Research Agent
## Task: Perform research to gather information for a blog post on Write a blog post about the importance of renewable energy..


# Agent: Research Agent
## Final Answer: 
**The Importance of Renewable Energy: A Sustainable Future**

As global energy demands soar and the impacts of climate change become increasingly evident, the importance of renewable energy has never been more pronounced. Transitioning to renewable energy sources is essential for creating a sustainable and resilient energy future. This blog post delves into key insights, facts, and trends surrounding renewable energy, emphasizing its significance in combating climate change, securing energy independence, and fostering economic growth.

### 1. Understanding Renewable Energy

Renewable energy refers to energy derived from natural processes that are replenished constantly. Key types of renewable energy include:

- **Solar Energy**: Harnessing sunlight through photovoltaic cells or solar thermal s